In [1]:
import polars as pl
import os

from fuzzywuzzy import fuzz

def fuzzy_match_ratio(s1: str, s2: str) -> int:

    if type(s1) != str or type(s2) != str:
        return -1
    
    # Normalize both strings to lower case for a case-insensitive match.
    s1, s2 = s1.lower(), s2.lower()
    # Calculate and return the fuzzy match ratio.
    return fuzz.ratio(s1, s2)


In [2]:
juvenile_case_mapping = {
    'Juvenile-Felony': 'Felony',
    'Juvenile-Misdemeanor/Infraction': 'Misdemeanor',
    'Juvenile-Traffic Offense': 'Traffic Offense',
    
    'Juvenile-Status Offender': 'Status',
    
    'Juvenile-Abandoned': 'Dependency',
    'Juvenile-Abused': 'Dependency',
    'Juvenile-Neglected': 'Dependency',
    'Juvenile-Parental Jurisdiction': 'Dependency',
    'Juvenile-Paternity Determination': 'Dependency',
    'Juvenile-Guard Disruptd/Terminatd-P2013': 'Dependency',
    'Juvenile-Guardianship Disruptd/Terminatd': 'Dependency',
    'Juvenile-Homeless/Destitute/No Fault': 'Dependency',
    'Juvenile-Termination': 'Dependency',
    'Juvenile-Termination/Divorce-P2013': 'Dependency',
    'Juvenile-Voluntary Relinquishment': 'Dependency',
    'Juvenile-Voluntary Relinquishment-P2013': 'Dependency',

    # Cases that may be administrative but likely fall under Dependency
    'Juvenile-Interstate Compact-Holding Stat': 'Dependency',  
    'Juvenile-Migrated Cases - Douglas ONLY': 'Dependency'  
}


In [3]:
df = pl.read_database_uri("SELECT * FROM CaseInformation", os.environ["database"])

In [5]:
df = df.filter( pl.col("FiledDate") != "").with_columns(
    pl.col("FiledDate").map_elements(lambda x: x[-4:]).alias("Year").cast(pl.Int64),
    pl.col("Classification").replace(juvenile_case_mapping)
).filter( pl.col("Year") > 2009).unique().filter( pl.col("Classification") != "Dependency" )

In [6]:
df_bi = pl.read_database_uri("SELECT * FROM CaseBasicInfo", os.environ["database"])

In [7]:
df_bi = df_bi.filter( pl.col("Type") == "JUV" ).select(["CaseID", "YearOfBirth", "Person"]).unique().with_columns(
    pl.col("Person").map_elements(lambda x: x.split(",")[0]).alias("LastName"),
    pl.col("Person").map_elements(lambda x: x.split(",")[1] if len(x.split(",")) > 1 else "").alias("FirstName")
).with_columns(
    pl.struct(["FirstName", "LastName"]).map_elements(lambda x: x["FirstName"] + " " + x["LastName"]).alias("Name")
).drop(["Person", "FirstName", "LastName"])

In [8]:
df_case_info = df.join(df_bi, on = "CaseID", how = "left").with_columns(
    pl.struct(["DefendantName", "Name"]).map_elements(lambda x: fuzzy_match_ratio(x["DefendantName"], x["Name"])).alias("MatchScore")
).sort( by = "MatchScore", descending=True ).unique(subset=["CaseID"], keep="first").with_columns(
    pl.struct(["DefendantName", "Name"]).map_elements(lambda x: x["Name"] if x["DefendantName"] == "" else x["DefendantName"]).alias("DefendantName")
).drop(["Name", "MatchScore"])

In [9]:
df_case_info = df_case_info.with_columns(
    pl.struct(["Year", "YearOfBirth"]).map_elements(lambda x: x["Year"] - x["YearOfBirth"] if type(x["YearOfBirth"]) is int and type(x["Year"]) is int else None).alias("Age")
).drop("DefendantType")

In [10]:
df_case_info.write_csv("./Data/JuvenileCaseInfo.csv")

In [ ]:
df_parties = pl.read_database_uri("SELECT * FROM Parties", os.environ["database"])
df_parties = df_parties.filter( pl.col("CaseID").is_in(df_case_info["CaseID"])).drop_nulls(subset = ["Role"]).drop("AttorneyName")
df_parties.write_csv("./Data/JuvenileInvolvedParties.csv")

In [ ]:
df_offenses = pl.read_database_uri("SELECT * FROM Offenses", os.environ["database"])
df_offenses = df_offenses.filter( pl.col("CaseID").is_in(df_case_info["CaseID"]) )
df_offenses.write_csv("./Data/JuvenileOffenses.csv")